In [28]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [29]:
print(len(text), len(set(text)))

1115394 65


In [30]:
chars = sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [31]:
itos = {i:char for i,char in enumerate(chars)}
stoi = {char:i for i, char in enumerate(chars)}
encode = lambda x: [stoi[char] for char in list(x)]
decode = lambda x: ''.join([itos[num] for num in list(x)])

print(encode('poor'), decode(encode("poor")))

[54, 53, 53, 56] poor


In [32]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [33]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [34]:
block_size = 8


In [35]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(context, target)

tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [36]:
torch.manual_seed(163173)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:block_size+i]for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    return x, y 

xb, yb = get_batch("train")
print(xb.shape)
print(yb.shape)
print(xb)
print(yb)

print("--------")
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"context: {context.tolist()}: target {target}")


torch.Size([4, 8])
torch.Size([4, 8])
tensor([[41, 53, 52, 57, 59, 51, 43, 42],
        [44,  1, 48, 53, 63,  0, 32, 46],
        [47, 43, 44,  1, 46, 39, 58, 46],
        [42,  1, 40, 43,  1, 46, 39, 52]])
tensor([[53, 52, 57, 59, 51, 43, 42,  1],
        [ 1, 48, 53, 63,  0, 32, 46, 39],
        [43, 44,  1, 46, 39, 58, 46,  1],
        [ 1, 40, 43,  1, 46, 39, 52, 45]])
--------
context: [41]: target 53
context: [41, 53]: target 52
context: [41, 53, 52]: target 57
context: [41, 53, 52, 57]: target 59
context: [41, 53, 52, 57, 59]: target 51
context: [41, 53, 52, 57, 59, 51]: target 43
context: [41, 53, 52, 57, 59, 51, 43]: target 42
context: [41, 53, 52, 57, 59, 51, 43, 42]: target 1
context: [44]: target 1
context: [44, 1]: target 48
context: [44, 1, 48]: target 53
context: [44, 1, 48, 53]: target 63
context: [44, 1, 48, 53, 63]: target 0
context: [44, 1, 48, 53, 63, 0]: target 32
context: [44, 1, 48, 53, 63, 0, 32]: target 46
context: [44, 1, 48, 53, 63, 0, 32, 46]: target 39
cont

In [39]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(163173)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.tocken_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets):
        logits = self.tocken_embedding_table(idx)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss
    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)
        

torch.Size([32, 65]) tensor(4.4245, grad_fn=<NllLossBackward0>)
